In [16]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from imblearn.combine import SMOTETomek
import mlflow
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Make imbalanced dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [4]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Experiment 1: Train logistic regression classifier

In [5]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94       180
           1       0.47      0.40      0.43        20

    accuracy                           0.90       200
   macro avg       0.70      0.68      0.69       200
weighted avg       0.89      0.90      0.89       200



# Experiment 2: Train random forest classifier

In [6]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth = 3)
rf_clf.fit(X_train, y_train)
y_pred_rf_clf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf_clf))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       180
           1       0.88      0.70      0.78        20

    accuracy                           0.96       200
   macro avg       0.92      0.84      0.88       200
weighted avg       0.96      0.96      0.96       200



# Experiment 3: Train XGBoost

In [7]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       180
           1       0.94      0.75      0.83        20

    accuracy                           0.97       200
   macro avg       0.96      0.87      0.91       200
weighted avg       0.97      0.97      0.97       200



# Experiment 4: class imbalance treated with SMOTETomek and then train XGBoost

In [11]:
smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([703, 703]))

In [13]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97       180
           1       0.70      0.80      0.74        20

    accuracy                           0.94       200
   macro avg       0.84      0.88      0.86       200
weighted avg       0.95      0.94      0.95       200



# MLFlow tracking

In [14]:
models = [
    (
        "Logistic Regression", 
        LogisticRegression(C=1, solver='liblinear'), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        RandomForestClassifier(n_estimators=30, max_depth=3), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [15]:
reports = []

for model_name, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [19]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection")
mlflow.set_tracking_uri("http://127.0.0.1:5000")

for i, element in enumerate(models):
    model_name = element[0]
    model = element[1]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_param("model", model_name)
        mlflow.log_metric('accuracy', report['accuracy'])
        mlflow.log_metric('recall_class_1', report['1']['recall'])
        mlflow.log_metric('precision_class_1', report['1']['precision'])
        mlflow.log_metric('recall_class_0', report['0']['recall'])
        mlflow.log_metric('precision_class_0', report['0']['precision'])
        mlflow.log_metric('f1_score_macro', report['macro avg']['f1-score'])        
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model") 

2025/10/11 17:48:51 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/11 17:48:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/10/11 17:48:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/605370573200176422/runs/b60a42d9900a4e8283cf5b1f319bbdaf
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/605370573200176422


2025/10/11 17:48:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/10/11 17:48:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/605370573200176422/runs/ed423e66f9e046f0a9ae64af1caa6669
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/605370573200176422


2025/10/11 17:48:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/10/11 17:48:57 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/605370573200176422/runs/4bcd84566446459aaab7ee75c4d1f3ee
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/605370573200176422


2025/10/11 17:48:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/605370573200176422/runs/a50cdecc77c94a0083fd89f34f7cdbe8
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/605370573200176422
